In [1]:
import pandas as pd
import geopandas as gpd
import random
import fiona
import matplotlib.pyplot as plt
from pathlib import Path
import random, os, sys
import numpy as np
import zipfile
import seaborn as sns

sys.path.insert(0, 'C:/Users/mmh/Documents/Codes/cross-sectro-transp-energy-model/')
# importlib.reload(sys.modules['src.calculation'])
from src.func_buildings import *
pd.set_option('display.max_columns', None)
data_dir = 'C:/Users/mmh/Documents/Data/'
data_maps = 'C:/Users/mmh/OneDrive - Oak Ridge National Laboratory/Melrose/8.Maps/'

# Arizona - Pima County
Original version: https://zenodo.org/records/7644412

Obviously after updaing the data set, they messed up the data: https://zenodo.org/record/8360283

## Building locations

In [39]:
building_loc = pd.read_csv(data_dir+'Buildings/AZ/AZ.csv', index_col=False)
print(len(building_loc))
building_loc[['latitude','longitude']] = building_loc.Centroid.str.split("/", expand = True)
building_loc = gpd.GeoDataFrame(building_loc, geometry=gpd.points_from_xy(building_loc.longitude, building_loc.latitude), 
                                crs=4269)    

2555144


In [40]:
building_loc[building_loc.ID==8019004583478]

,ID,Centroid,Footprint2D,State,Area,Area2D,Height,NumFloors,WWR_surfaces,CZ,BuildingType,Standard,latitude,longitude,geometry


In [41]:
building_loc.head(2)

,ID,Centroid,Footprint2D,State,Area,Area2D,Height,NumFloors,WWR_surfaces,CZ,BuildingType,Standard,latitude,longitude,geometry
0,8100000000000,36.9250485/-113.93949299999997,36.92509342519237/-113.939476279076_36.9250399...,AZ,302.21482,302.21482,9.0,1.0,.14_.14_.14_.14,3B,IECC,90.1-2010,36.9250485,-113.93949299999997,POINT (-113.93949 36.92505)
1,8100000000001,36.917147500000006/-113.93760900000001,36.91725884689579/-113.9376613924597_36.917258...,AZ,1745.59563,1745.59563,9.0,1.0,.14_.14_.14_.14,3B,IECC,90.1-2010,36.917147500000006,-113.93760900000001,POINT (-113.93761 36.91715)


In [3]:
building_loc.BuildingType.value_counts()

IECC                      2314428
SmallOffice                 93679
RetailStandalone            40194
MediumOffice                21096
RetailStripmall             18986
MidriseApartment            12092
Outpatient                  10500
Warehouse                   10426
FullServiceRestaurant        8423
QuickServiceRestaurant       7198
SmallHotel                   5527
SecondarySchool              4272
PrimarySchool                2902
HighriseApartment            2371
LargeOffice                  1839
LargeHotel                    706
Hospital                      505
Name: BuildingType, dtype: int64

In [21]:
AZ_shp = gpd.read_file(data_maps+'geo/tl_2019/tl_2019_04_bg.zip', index_col=False)
building_loc = building_loc.sjoin(AZ_shp, how="left", predicate="within")
# building_loc.to_file('AZ_buildings.shp', driver='ESRI Shapefile')

## Select all the buildings in PC
building_loc = building_loc[building_loc.COUNTYFP == '019'] # Pima County

## Select all the buildings in three BGS
building_loc = building_loc[((building_loc.TRACTCE=='000500')&(building_loc.BLKGRPCE=='2'))| 
                             ((building_loc.TRACTCE=='000500')&(building_loc.BLKGRPCE=='4'))|
                             ((building_loc.TRACTCE=='000400')&(building_loc.BLKGRPCE=='2'))]
# buildings_pima_selected = buildings_pima_selected.sample(frac=0.5) # randomly select 20% of the buildings
print(len(building_loc))

building_loc['BuildingClass'] = 'Commercial'
building_loc.loc[building_loc.BuildingType.isin(['MidriseApartment','HighriseApartment','IECC']),'BuildingClass'] = 'Residential'
building_loc.loc[building_loc.BuildingType.isin(['SmallOffice','MediumOffice','LargeOffice',
                                              'PrimarySchool','SecondarySchool']),'BuildingClass'] = 'Industrial'

%store building_loc

building_loc.head()

877
Stored 'building_loc' (GeoDataFrame)


,ID,Centroid,Footprint2D,State,Area,Area2D,Height,NumFloors,WWR_surfaces,CZ,BuildingType,Standard,latitude,longitude,geometry,index_right,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,BuildingClass
361964,8100000361968,32.2332017822464/-110.97001163562747,32.23326743830728/-110.9700761553153_32.233267...,AZ,1440.102037,1440.102037,16.092549,1.0,.14_.14_.14_.14_.14_.14,2B,IECC,DOE-Ref-1980-2004,32.2332017822464,-110.97001163562747,POINT (-110.97001 32.23320),3035,04,019,000400,2,040190004002,Block Group 2,G5030,S,639759,0,+32.2359665,-110.9668684,Residential
362043,8100000362047,32.222776094509754/-110.9542671434197,32.22283314041453/-110.9543221547136_32.222833...,AZ,1320.563362,1320.563362,10.966123,1.0,.14_.14_.14_.14_.14_.14,2B,IECC,DOE-Ref-Pre-1980,32.222776094509754,-110.9542671434197,POINT (-110.95427 32.22278),146,04,019,000500,2,040190005002,Block Group 2,G5030,S,213479,0,+32.2235080,-110.9475650,Residential
362104,8100000362108,32.2347135/-110.962099,32.23475231954001/-110.9621434254698_32.234752...,AZ,475.575940,475.575940,13.123360,1.0,.14_.14_.14_.14,2B,IECC,DOE-Ref-1980-2004,32.2347135,-110.962099,POINT (-110.96210 32.23471),3035,04,019,000400,2,040190004002,Block Group 2,G5030,S,639759,0,+32.2359665,-110.9668684,Residential
362976,8100000362980,32.234692/-110.96525199999999,32.23472110055219/-110.9652826223493_32.234721...,AZ,185.008727,185.008727,12.354517,1.0,.14_.14_.14_.14,2B,IECC,DOE-Ref-1980-2004,32.234692,-110.96525199999999,POINT (-110.96525 32.23469),3035,04,019,000400,2,040190004002,Block Group 2,G5030,S,639759,0,+32.2359665,-110.9668684,Residential
363188,8100000363192,32.237486000000004/-110.96174350000001,32.23755266774356/-110.9618089771458_32.237552...,AZ,1407.398541,1407.398541,12.587862,1.0,.14_.14_.14_.14,2B,IECC,DOE-Ref-Pre-1980,32.237486000000004,-110.96174350000001,POINT (-110.96174 32.23749),3035,04,019,000400,2,040190004002,Block Group 2,G5030,S,639759,0,+32.2359665,-110.9668684,Residential


In [28]:
building_loc.to_file('3BG_buildings.shp', driver='ESRI Shapefile')

C:\Users\mmh\AppData\Local\Temp\ipykernel_2328\4091208752.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  building_loc.to_file('3BG_buildings.shp', driver='ESRI Shapefile')


In [25]:
building_loc.BuildingType.value_counts()

IECC                     776
SmallOffice               17
MediumOffice              16
LargeHotel                12
RetailStandalone           8
Hospital                   7
RetailStripmall            6
Outpatient                 6
Warehouse                  6
MidriseApartment           6
HighriseApartment          6
LargeOffice                5
SecondarySchool            3
PrimarySchool              1
FullServiceRestaurant      1
SmallHotel                 1
Name: BuildingType, dtype: int64

In [32]:
building_loc[building_loc.ID=='8019004583500']

,ID,Centroid,Footprint2D,State,Area,Area2D,Height,NumFloors,WWR_surfaces,CZ,BuildingType,Standard,latitude,longitude,geometry


## Building capacity

In [27]:
df = pd.DataFrame()
# buildingType = pd.DataFrame()
i = 0
for fname in filenames:
    loc_fname = os.path.join(loc, fname)
    if Path(loc_fname).is_file():
        x = read_idf(loc_fname)
        x['ID'] = fname
        if len(type_subset)!=0 and len(name_subset)!=0:
            x = x[(x.type.isin(type_subset)) & (x.name.isin(name_subset))]
        elif len(type_subset)!=0 and len(name_subset)==0:
            x = x[(x.type.isin(type_subset))]
        df = pd.concat([df,x])
    i = i + 1
        # buildingType = pd.concat([buildingType,xType])
df.head()      
# df['ID'] = df['ID'].str.replace(r'.idf', '', regex=True)
# df['ID'] = df['ID'].astype('int64')

# print('size: '+str(round(df.memory_usage(index=True).sum()/1e9,4))+'GB')

""


In [24]:
loc = data_dir+'Buildings/IDF_AZ/AZ_Pima'
type_subset = ['People']
# filenames = random.sample(os.listdir(loc), 5)
filenames = building_loc.ID.astype(str)+'.idf'
building_set = process_idf(loc,filenames,type_subset)
building_set = pd.merge(building_set,building_loc[['ID','TRACTCE','BLKGRPCE']],how='left',on='ID')
building_set.head()

KeyError: 'ID'

In [ ]:
set(building_set.name)

{'Activity Level Schedule Name',
 'Air Velocity Schedule Name',
 'Carbon Dioxide Generation Rate {m3/s-W}',
 'Clothing Insulation Calculation Method',
 'Clothing Insulation Calculation Method Schedule Name',
 'Clothing Insulation Schedule Name',
 'Enable ASHRAE 55 Comfort Warnings',
 'Fraction Radiant',
 'Mean Radiant Temperature Calculation Type',
 'Name',
 'Number of People',
 'Number of People Calculation Method',
 'Number of People Schedule Name',
 'People per Zone Floor Area {person/m2}',
 'Sensible Heat Fraction',
 'Surface Name/Angle Factor List Name',
 'Work Efficiency Schedule Name',
 'Zone Floor Area per Person {m2/person}',
 'Zone or ZoneList Name',
 nan}

In [ ]:
# examine: some buildings do not have this information - may not need to simulate?
inclusion_ID = building_set[building_set.name=='People per Zone Floor Area {person/m2}'].ID 
len(building_loc[~building_loc.ID.isin(inclusion_ID)])

52

In [ ]:
selected_cols = ['ID','TRACTCE','BLKGRPCE','Area','latitude','longitude']
building_cap = pd.merge(building_set,building_loc[selected_cols],how='left',on=['ID','TRACTCE','BLKGRPCE'])

### need to double check whether this parameter is used
building_cap = building_cap[building_cap.name=='People per Zone Floor Area {person/m2}'] # some buildings do not have this information - may not need to simulate?
# building_cap[building_cap.isna().any(axis=1)]
building_cap['capacity'] = building_cap['value'].astype(float) * building_cap['Area']
building_cap = building_cap[selected_cols+['capacity']]
print(len(set(building_cap.ID)))

%store building_cap

building_cap.head()

849
Stored 'building_cap' (DataFrame)


,ID,TRACTCE,BLKGRPCE,Area,latitude,longitude,capacity
5,8019004624875,000400,2,4320.306112,32.2332017822464,-110.97001163562747,77.505678
25,8019004624951,000500,2,2641.126724,32.222776094509754,-110.9542671434197,47.381438
45,8019004625008,000400,2,951.151879,32.2347135,-110.962099,17.063530
65,8019004625839,000400,2,370.017454,32.234692,-110.96525199999999,6.638061
85,8019004626038,000400,2,2814.797081,32.237486000000004,-110.96174350000001,50.497060


In [ ]:
building_cap[building_cap.isna().any(axis=1)]

,ID,TRACTCE,BLKGRPCE,Area,latitude,longitude,capacity


## Building types from IDF files

In [ ]:
# loc = data_dir+'Buildings/IDF_AZ/AZ_Pima'
# type_subset = ['Building']
# name_subset = ['Name']
# filenames = building_loc.ID.astype(str)+'.idf'
# buildingType = process_idf(loc,filenames,type_subset,name_subset)

# buildingType['BuildingType_idf'] = buildingType['value'].str.split('-').str[1]
# buildingType.BuildingType_idf.value_counts()

size: 0.0GB


MidriseApartment          815
SmallOffice                21
RetailStandalone           21
Warehouse                  16
SecondarySchool             7
RetailStripmall             5
PrimarySchool               5
Outpatient                  4
FullServiceRestaurant       3
QuickServiceRestaurant      2
LargeHotel                  1
Name: BuildingType_idf, dtype: int64

In [ ]:
# type_comparison = pd.merge(building_loc,buildingType[['ID','BuildingType_idf']],how='left',on=['ID'])
# type_comparison = type_comparison[['ID','BuildingType','BuildingType_idf']]
# type_comparison = type_comparison[(type_comparison.BuildingType!='IECC') & (type_comparison.BuildingType!=type_comparison.BuildingType_idf)]
# type_comparison.to_csv(data_dir+'Buildings/'+'building_types.csv',index=False)
# type_comparison.head()

,ID,BuildingType,BuildingType_idf
4,8019004626038,SmallOffice,MidriseApartment
10,8019004629838,LargeHotel,Warehouse
22,8019004630841,SmallOffice,MidriseApartment
57,8019004637601,HighriseApartment,MidriseApartment
76,8019004642804,FullServiceRestaurant,MidriseApartment


# Building set - Default schedules

In [ ]:
# type_subset = ['Schedule:Day:Interval']
# loc = data_dir+'Buildings/IDF_AZ/AZ_Pima'
# filenames = building_loc.ID.astype(str)+'.idf'
# building_set = process_idf(loc,filenames,type_subset)
# %store building_set
# print(len(np.unique(building_set.ID)))

size: 0.0201GB
Stored 'building_set' (DataFrame)
900
